In [1]:
import numpy as np
import pandas as pd

import os
import glob
import shutil
import pickle
import random
from collections import Counter

from sklearn.utils import shuffle

import tensorflow as tf
from keras import models, layers, optimizers
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16, ResNet50, InceptionResNetV2
from tensorflow.keras.applications import EfficientNetB3
from keras.utils import to_categorical
from keras.losses import CategoricalCrossentropy
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, GlobalAveragePooling2D, average
from keras.optimizers import RMSprop, Adam, SGD
from keras.callbacks import (ModelCheckpoint, EarlyStopping, 
                                        ReduceLROnPlateau)
from tensorflow.keras.experimental import CosineDecay
from keras import Model
from PIL import Image

from keras import backend as K
import gc
from sklearn.model_selection import KFold

from itertools import combinations  
from tqdm.notebook import tqdm

In [2]:
# Set Seed for reproducible results
seed_value= 0

os.environ['PYTHONHASHSEED']=str(seed_value)

random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, 
                                        inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), 
                            config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

### Import Data

In [4]:
# Defining the working directories

work_dir = 'cassava-leaf-disease-classification/'
os.listdir(work_dir) 
train_path = 'cassava-leaf-disease-classification/train_images/'

In [5]:
mapping = pd.read_json(work_dir + 'label_num_to_disease_map.json', 
    lines=True
).transpose()[0].to_dict()
mapping

{0: 'Cassava Bacterial Blight (CBB)',
 1: 'Cassava Brown Streak Disease (CBSD)',
 2: 'Cassava Green Mottle (CGM)',
 3: 'Cassava Mosaic Disease (CMD)',
 4: 'Healthy'}

In [6]:
data = pd.read_csv(work_dir + '/train.csv')
data = shuffle(data, random_state=42)
data["filepath"] = train_path+data["image_id"]

# split data into out-of-fold (oof) and training (df)
oof_percentage = 0.2
oof_count = int(len(data)*0.2)
oof = data[:oof_count]
df = data[oof_count:]

oof.reset_index(inplace=True)
df.reset_index(inplace=True)

# sanity check
print(f'oof: {len(oof)}')
print(f'df: {len(df)}')

oof: 4279
df: 17118


In [6]:
BATCH_SIZE = 8
image_size = 512
input_shape = (image_size, image_size, 3)
dropout_rate = 0.4
classes_to_predict = 5

In [7]:
data_augmentation_layers = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.RandomCrop(height=image_size, width=image_size),
        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        layers.experimental.preprocessing.RandomRotation(0.25),
        layers.experimental.preprocessing.RandomZoom((-0.2, 0)),
        layers.experimental.preprocessing.RandomContrast((0.2,0.2))
    ]
)

## Model Training

#### Create Model

In [8]:
augmentation = 'cassava-data-augmentation/augmentation'
original = 'cassava-leaf-disease-classification/train_images'

EPOCHS = 8

def scce_with_ls(y, y_hat):
    y = tf.one_hot(tf.cast(y, tf.int32), 5)
    y = tf.squeeze(y, axis=1)
    return categorical_crossentropy(y, y_hat, label_smoothing=0.3)

LOSS = scce_with_ls

In [9]:
def create_model():
    conv_base = EfficientNetB3(weights='imagenet',
                      include_top=False,
                      input_shape=(input_shape)
                     )

    inputs = Input(shape=input_shape)
    augmented = data_augmentation_layers(inputs)
    conv_base = conv_base(augmented)
    pooling = layers.GlobalAveragePooling2D()(conv_base)
    dropout = layers.Dropout(dropout_rate)(pooling)
    outputs = Dense(classes_to_predict, activation="softmax")(dropout)
    model = Model(inputs=inputs, outputs=outputs)

    decay_steps = int(round(len(df)*0.8)/BATCH_SIZE)*EPOCHS
    cosine_decay = CosineDecay(initial_learning_rate=1e-4,
                               decay_steps=decay_steps, alpha=0.3)

    model.compile(loss=LOSS,
                  optimizer=tf.keras.optimizers.Adam(cosine_decay), 
                  metrics=["accuracy"])
    
    return model

#### Cross Validation Training

In [10]:
def load_image_and_label_from_path(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    return img, label

In [11]:
def train_cross_validate(folds=5):
    histories = []
    models = []
    
    kfold = KFold(folds, shuffle=True, random_state=42)
    
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(df)):
        
        # Free up Memory
        gc.collect()
        K.clear_session()
        
        print(f'\nFold: {fold}')
        
        # Load datasets
        training_df = df.loc[trn_ind]
        validation_df = df.loc[val_ind]
       
        training_data = tf.data.Dataset.from_tensor_slices(
            (training_df.filepath.values, training_df.label.values))
        validation_data = tf.data.Dataset.from_tensor_slices(
            (validation_df.filepath.values, validation_df.label.values))
        
        AUTOTUNE = tf.data.experimental.AUTOTUNE
        
        training_data = training_data.map(
            load_image_and_label_from_path, num_parallel_calls=AUTOTUNE)
        validation_data = validation_data.map(
            load_image_and_label_from_path, num_parallel_calls=AUTOTUNE)
        
        training_data_batches = (training_data
                                 .shuffle(buffer_size=1000)
                                 .batch(BATCH_SIZE)
                                 .prefetch(buffer_size=AUTOTUNE))
        validation_data_batches = (validation_data
                                   .shuffle(buffer_size=1000)
                                   .batch(BATCH_SIZE)
                                   .prefetch(buffer_size=AUTOTUNE))
        
        
        # Setup Callbacks
        checkpoint_name = 'efficientnetb3-CV-ls_0.3_oof_'+str(fold)+'.h5'        
        callbacks = [ModelCheckpoint(
            filepath=checkpoint_name, 
            monitor='val_loss', 
            save_best_only=True)]
        
        # Get model and start training
        model = create_model()      
        history = model.fit(training_data_batches,
                            epochs = EPOCHS, 
                            validation_data=validation_data_batches,
                            callbacks=callbacks, 
                            batch_size=BATCH_SIZE)
        
        print('Load best weights for model prediction')
        model.load_weights(checkpoint_name)
        models.append(model)
        histories.append(history)

    return histories, models

In [12]:
histories, models = train_cross_validate()


Fold: 0
Epoch 1/8
1712/1712 [==============================] - 1018s 583ms/step - loss: 1.2157 - accuracy: 0.7343 - val_loss: 1.0637 - val_accuracy: 0.8659
Epoch 2/8
1712/1712 [==============================] - 1001s 583ms/step - loss: 1.0837 - accuracy: 0.8503 - val_loss: 1.0455 - val_accuracy: 0.8744
Epoch 3/8
1712/1712 [==============================] - 999s 582ms/step - loss: 1.0595 - accuracy: 0.8685 - val_loss: 1.0372 - val_accuracy: 0.8855
Epoch 4/8
1712/1712 [==============================] - 996s 580ms/step - loss: 1.0464 - accuracy: 0.8756 - val_loss: 1.0348 - val_accuracy: 0.8817
Epoch 5/8
1712/1712 [==============================] - 998s 581ms/step - loss: 1.0321 - accuracy: 0.8877 - val_loss: 1.0368 - val_accuracy: 0.8841
Epoch 6/8
1712/1712 [==============================] - 1000s 582ms/step - loss: 1.0183 - accuracy: 0.8978 - val_loss: 1.0347 - val_accuracy: 0.8823
Epoch 7/8
1712/1712 [==============================] - 999s 582ms/step - loss: 1.0099 - accuracy: 0.9043 -